In [146]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold

boston=load_boston
X_data,t=load_boston(return_X_y=True) 
# split the data into training and validation set   
X_train, X_valid, t_train, t_valid = train_test_split(X_data,t,test_size=1/5, random_state=10) 

In [147]:
N = len(X_train)
M = len(X_valid)


# function of getting the predication y; Input = X, t; Output = y, w
def getRMSE(X_train,X_valid,t_train,t_valid):
    new_col = np.ones(len(X_train))
    Xtr = np.column_stack((new_col,X_train))    # add a column of one
    A = np.dot(Xtr.T,Xtr)
    A1 = np.linalg.inv(A) #the inverse of A
    B = np.dot(Xtr.T,t_train)
    w =np.dot(A1,B)
    y = np.dot(Xtr,w)
    
    # prepare the validation set
    new_col_1 = np.ones(len(X_valid))
    Xva = np.column_stack((new_col_1, X_valid))
    y_valid = np.dot(Xva,w)
    diff_valid = np.subtract(t_valid,y_valid)
    err_valid = np.dot(diff_valid, diff_valid)/len(X_valid)
    RMSE_valid = np.sqrt(err_valid)

    return RMSE_valid


In [148]:
def kFold(splits, X, t):
    res = 0
    kf=KFold(n_splits=splits)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        t_train, t_test = t[train_index], t[test_index]

        res = res + getRMSE(X_train,X_test,t_train,t_test)
  
    error = res/splits

    return error    

In [149]:
K=13
S = np.ones((N,1))  # dummy ones
S_index = []    # store the orginal column number
error = np.zeros((K,1))
existingCol = np.arange(K)

for i in range(K):
    
    #print(existingCol)
    if (i==0):
        for j in existingCol:

            X_train_temp=X_train[:,j]  # only select a column for the first run  
            error[j] = kFold(5, X_train_temp, t_train)
            S = X_train[:,np.argmin(error)]

        existingCol_new =np.delete(existingCol,np.where(existingCol == np.argmin(error) ))
        existingCol = existingCol_new
        error = np.full((K,1),np.inf)

    else:
        for j in existingCol:
            
            X_train_temp = np.column_stack( (S,np.reshape(X_train[:,j], (len(X_train), 1))) ) #######
            error[j] = kFold(5, X_train_temp, t_train)
 
        S = np.column_stack( (S,  np.reshape(X_train[:,np.argmin(error)], (len(X_train), 1))) )
        existingCol_new =np.delete(existingCol,np.where(existingCol == np.argmin(error) ))
        existingCol = existingCol_new
        error = np.full((K,1),np.inf)


print(S.shape)
        



(404, 13)
